In [ ]:
import numpy as np
import matplotlib.pyplot as plt

from numpy.linalg import inv

In [ ]:
from itertools import product
from lib.total_variation import TotalVariation
from models.lib.firstorderode.base import RungeKutta
from models.lib.firstorderode.roessler import Roessler76
from models.lib.firstorderode.sprott import SprottAttractors
from models.lib.toy_models import LinaJL
from reservoir_helper import get_lorenz63_data_and_reservoir

models = []
for ODEClass in [* SprottAttractors[1:4], Roessler76]:
    models += lambda seed = None, n_buffer = 10000, ODEClass=ODEClass: RungeKutta(ODEClass(seed), 2e-2, 10, n_buffer),
models += LinaJL,

model_names = ["SprottB", "SprottC", "SprottD", "Roessler76", "Lorenz63"]

In [ ]:
from scipy.stats import chi2
chi2 = chi2(10)

In [ ]:
fontsize=12
plt.rcParams.update({'font.size': fontsize})

for dt_02 in (True, False):

    n_nodes = 20
    quantiles = np.load(f"results/quantiles{"" if not dt_02 else "_02"}.npy")

    for i_model, Model in enumerate(models):
        fig, axs = plt.subplots(6, 2, figsize=(10, 12))
        data = np.load(f"results/spectral_radius_n_nodes_{n_nodes}/uncoupled{"" if not dt_02 else "_dt_02"}_{Model(0,0).ode.__class__.__name__}.npz")

        n_linspace = 100
        names = data["names"][0, 0]
        measures = data["measures"][0].T
        name2index = { name:i for i, name in enumerate(names)}
        name2index["idx"] = 0
        n_samples = measures.shape[1] // n_linspace
        idx_bnd_osc = np.logical_and(measures[name2index["bounded"]], measures[name2index["ozillating"]]).reshape(n_linspace, n_samples) #measures[name2index["bounded"]].reshape(n_linspace, n_samples).astype(int) #
        for i_measure, (i_measure_quantile, name, sname) in enumerate(zip((None, None, 3, 1, 2, 0), ('valid_prediciton_time', "idx", 'Heikki', 'Adev', 'Ddev', 'AIncs'),  ("VPT", "#Bounded & Oscillating", "GCI", "ADev", "TVar", "AExc"))):
            ax = axs[i_measure,0 ]
            ax.set_ylabel(sname)

            ax2 = axs[i_measure, 1]
            ax2.set_ylabel(sname)

            if i_measure == 0:
                ax2.set_title("Rejection rate")
                ax.set_title("Value")

            median = np.zeros(100) 
            median2 = np.zeros(100) 
            error_down, error_upper = np.zeros(100), np.zeros(100) 
            error_down2, error_upper2 = np.zeros(100), np.zeros(100) 
            for i, (d, ia) in enumerate(zip(measures[name2index[name]].reshape(100, n_samples), idx_bnd_osc)):
                if sum(ia) == 0:
                    median[i] = np.nan
                    error_down[i] = np.nan
                    error_upper[i] = np.nan
                    continue
                if name == "valid_prediciton_time":
                    d *= 0.1 if not dt_02 else 0.02
                if name == "AIncs":
                    d = 1 - d
                if name == "mse":
                    median[i] = median2[i] = np.median(d[ia])
                    error_down[i] = error_down2[i] = np.quantile(d[ia], 0.25)
                    error_upper[i] = error_upper2[i] = np.quantile(d[ia], 0.75)
                elif name == "valid_prediciton_time":
                    median[i] = median2[i] = np.median(d)
                    error_down[i] = error_down2[i] = np.quantile(d, 0.25)
                    error_upper[i] = error_upper2[i] = np.quantile(d, 0.75)
                elif name == "idx":
                    median2[i] = median[i] = np.mean(ia)
                    error_down2[i] = error_down[i] = median[i] 
                    error_upper2[i] = error_upper[i] = median[i] 
                else:
                    if name == "Heikki":
                        threshold = chi2.isf(0.05)
                    else:
                        threshold = quantiles[-1, i_model, i_measure_quantile]
                    test_statistic = (threshold < d[ia])
                    median2[i] = np.mean(test_statistic)
                    var = np.var(test_statistic)
                    error_down2[i] = np.mean(test_statistic) - var
                    error_upper2[i] = np.mean(test_statistic) + var

                    median[i] = np.median(d[ia])
                    error_down[i] = np.quantile(d[ia], 0.25)
                    error_upper[i] = np.quantile(d[ia], 0.75)
                    
                    
            ax.set_xlabel("spectral radius")
            x = np.linspace(0, 1.5, 100)
            fig.subplots_adjust(hspace=0.25, wspace=0.30)
            ax.fill_between(x, error_down, error_upper, color='gray', alpha=0.5, label='Error margin')
            ax.plot(x, median)

            ax2.fill_between(x, error_down2, error_upper2, color='gray', alpha=0.5, label='Error margin')
            ax2.plot(x, median2)
        plt.savefig(f"pictures/supplemental/spectral_radius_all_dt_{"" if not dt_02 else "_02"}_{Model(0,0).ode.__class__.__name__}.pdf")
        plt.show() 

In [ ]:
fontsize=12
plt.rcParams.update({'font.size': fontsize})

for dt_02 in (True, False):

    n_nodes = 20
    quantiles = np.load(f"results/quantiles{"" if not dt_02 else "_02"}.npy")

    for i_model, Model in enumerate(models):
        fig, axs = plt.subplots(6, 1, figsize=(10, 12))
        data = np.load(f"results/spectral_radius_n_nodes_{n_nodes}/uncoupled{"" if not dt_02 else "_dt_02"}_{Model(0,0).ode.__class__.__name__}.npz")

        n_linspace = 100
        names = data["names"][0, 0]
        measures = data["measures"][0].T
        name2index = { name:i for i, name in enumerate(names)}
        name2index["idx"] = 0
        n_samples = measures.shape[1] // n_linspace
        idx_bnd_osc = np.logical_and(measures[name2index["bounded"]], measures[name2index["ozillating"]]).reshape(n_linspace, n_samples) #measures[name2index["bounded"]].reshape(n_linspace, n_samples).astype(int) #
        for i_measure, (i_measure_quantile, name, sname) in enumerate(zip((None, None, 3, 1, 2, 0), ('valid_prediciton_time', "idx", 'Heikki', 'Adev', 'Ddev', 'AIncs'),  ("VPT", "#Bounded & Oscillating", "GCI", "ADev", "TVar", "AExc"))):
            ax = axs[i_measure]
            ax.set_ylabel(sname)
            median = np.zeros(100) 
            error_down, error_upper = np.zeros(100), np.zeros(100) 
            for i, (d, ia) in enumerate(zip(measures[name2index[name]].reshape(100, n_samples), idx_bnd_osc)):
                if sum(ia) == 0:
                    median[i] = np.nan
                    error_down[i] = np.nan
                    error_upper[i] = np.nan
                    continue
                if name == "valid_prediciton_time":
                    d *= 0.1 if not dt_02 else 0.02
                if name == "AIncs":
                    d = 1 - d
                if name == "mse":
                    median[i] = np.median(d[ia])
                    error_down[i] = np.quantile(d[ia], 0.25)
                    error_upper[i] = np.quantile(d[ia], 0.75)
                elif name == "valid_prediciton_time":
                    median[i] = np.median(d)
                    error_down[i] = np.quantile(d, 0.25)
                    error_upper[i] = np.quantile(d, 0.75)
                elif name == "idx":
                    median[i] = np.mean(ia)
                    error_down[i] = median[i] 
                    error_upper[i] = median[i] 
                else:
                    if name == "Heikki":
                        threshold = chi2.isf(0.05)
                    else:
                        threshold = quantiles[-1, i_model, i_measure_quantile]
                    test_statistic = (threshold < d[ia])
                    median[i] = np.mean(test_statistic)
                    var = np.var(test_statistic)
                    error_down[i] = np.mean(test_statistic) - var
                    error_upper[i] = np.mean(test_statistic) + var
                    
            ax.set_xlabel("spectral radius")
            x = np.linspace(0, 1.5, 100)
            fig.subplots_adjust(hspace=0.25, wspace=0.30)
            ax.fill_between(x, error_down, error_upper, color='gray', alpha=0.5, label='Error margin')
            ax.plot(x, median)
        plt.savefig(f"pictures/supplemental/spectral_radius_threshold_dt_{"" if not dt_02 else "_02"}_{Model(0,0).ode.__class__.__name__}.pdf")
        plt.show() 

In [ ]:

n_nodes = 20
quantiles = np.load(f"results/quantiles{"" if not dt_02 else "_02"}.npy")

for dt_02 in (True, False):
    for i_model, Model in enumerate(models):
        fig, axs = plt.subplots(6, 1, figsize=(10, 12))
        data = np.load(f"results/spectral_radius_n_nodes_{n_nodes}/uncoupled{"" if not dt_02 else "_dt_02"}_{Model(0,0).ode.__class__.__name__}.npz")

        n_linspace = 100
        names = data["names"][0, 0]
        measures = data["measures"][0].T
        name2index = { name:i for i, name in enumerate(names)}
        name2index["idx"] = 0
        n_samples = measures.shape[1] // n_linspace
        idx_bnd_osc = np.logical_and(measures[name2index["bounded"]], measures[name2index["ozillating"]]).reshape(n_linspace, n_samples) #measures[name2index["bounded"]].reshape(n_linspace, n_samples).astype(int) #
        for i_measure, (i_measure_quantile, name, sname) in enumerate(zip((None, None, 3, 1, 2, 0), ('valid_prediciton_time', "idx", 'Heikki', 'Adev', 'Ddev', 'AIncs'), ("VPT", "#Bounded & Oscillating", "Log GCI", "ADev", "TVar", "AExc"))):
            ax = axs[i_measure]
            plt.ticklabel_format(axis='y', style='sci', scilimits=(0,0))
            ax.set_ylabel(sname)
            median = np.zeros(100) 
            error_down, error_upper = np.zeros(100), np.zeros(100) 
            for i, (d, ia) in enumerate(zip(measures[name2index[name]].reshape(100, n_samples), idx_bnd_osc)):
                if sum(ia) == 0:
                    median[i] = np.nan
                    error_down[i] = np.nan
                    error_upper[i] = np.nan
                    continue
                if name == "valid_prediciton_time":
                    d *= 0.1 if not dt_02 else 0.02
                if name == "AIncs":
                    d = 1 - d
                if name == "mse":
                    median[i] = np.median(d[ia])
                    error_down[i] = np.quantile(d[ia], 0.25)
                    error_upper[i] = np.quantile(d[ia], 0.75)
                elif name == "valid_prediciton_time":
                    median[i] = np.median(d)
                    error_down[i] = np.quantile(d, 0.25)
                    error_upper[i] = np.quantile(d, 0.75)
                elif name == "idx":
                    median[i] = np.mean(ia)
                    error_down[i] = median[i] 
                    error_upper[i] = median[i] 
                else:
                    if name == "Heikki":
                        threshold = np.log(chi2.isf(0.05))
                        d = np.log(d)
                        #ax.set_yscale("log")
                    else:
                        threshold = quantiles[-1, i_model, i_measure_quantile]
                    median[i] = np.median(d[ia])
                    error_down[i] = np.quantile(d[ia], 0.25)
                    error_upper[i] = np.quantile(d[ia], 0.75)
                    

            ax.set_xlabel("spectral radius")
            x = np.linspace(0, 1.5, 100)
            #fig.subplots_adjust(hspace=0.25, wspace=0.30)
            ax.fill_between(x, error_down, error_upper, color='gray', alpha=0.5, label='Error margin')
            ax.plot(x, median)
        plt.savefig(f"pictures/supplemental/spectral_radius_dt{"" if not dt_02 else "_02"}_{Model(0, 0).ode.__class__.__name__}.pdf")
        plt.show()

In [ ]:
cm = 1/2.54
fig, axs = plt.subplots(2, 4, figsize=((21 - 5)*cm*4/5, 3), sharey='row', sharex=True)
dt_02 = True
n_nodes = 20
fontsize = 9
plt.rcParams.update({'font.size': fontsize})
quantiles = np.load(f"results/quantiles{"" if not dt_02 else "_02"}.npy")
i_model = 4
Model = models[i_model]
data = np.load(f"results/spectral_radius_n_nodes_{n_nodes}/uncoupled{"" if not dt_02 else "_dt_02"}_{Model(0,0).ode.__class__.__name__}.npz")

n_linspace = 100
names = data["names"][0, 0]
measures = data["measures"][0].T
name2index = { name:i for i, name in enumerate(names)}
name2index["idx"] = 0
n_samples = measures.shape[1] // n_linspace
idx_bnd_osc = np.logical_and(measures[name2index["bounded"]], measures[name2index["ozillating"]]).reshape(n_linspace, n_samples) #measures[name2index["bounded"]].reshape(n_linspace, n_samples).astype(int) #
for i_measure, (i_measure_quantile, name, sname) in enumerate(zip((3, 1, 2, 0), ('Heikki', 'Adev', 'Ddev', 'AIncs'), ("Log GCI", "ADev", "TVar", "AExc"))):
    ax = axs[0, i_measure]
    ax.set_title(sname, fontsize=fontsize)

    median = np.zeros(100) 
    error_down, error_upper = np.zeros(100), np.zeros(100) 
    for i, (d, ia) in enumerate(zip(measures[name2index[name]].reshape(100, n_samples), idx_bnd_osc)):
        if sum(ia) == 0:
            median[i] = np.nan
            error_down[i] = np.nan
            error_upper[i] = np.nan
            continue

        if name == "AIncs":
            d = 1 - d

        #mi, ma = np.min(d[ia]), np.max(d[ia])
        #d = (d - mi) / (ma - mi)

        if name == "Heikki":
            threshold = chi2.isf(0.05)
        else:
            threshold = quantiles[-1, i_model, i_measure_quantile]

        if name == "AIncs":
            d = 1 - d
            threshold = 1 - threshold
        test_statistic = (threshold < d[ia])
        median[i] = np.mean(test_statistic)
        var = np.var(test_statistic)
        error_down[i] = np.mean(test_statistic) - var
        error_upper[i] = np.mean(test_statistic) + var

    ax.set_xlabel("spectral radius", fontsize=fontsize)
    x = np.linspace(0, 1.5, 100)
    ax.fill_between(x, error_down, error_upper, color='gray', alpha=0.5)
    axs[0, 0].set_ylabel("Rejected in %", fontsize=fontsize)
    ax.plot(x, median)


    # plot lower row
    ax = axs[1, i_measure]

    median = np.zeros(100) 
    error_down, error_upper = np.zeros(100), np.zeros(100) 
    for i, (d, ia) in enumerate(zip(measures[name2index[name]].reshape(100, n_samples), idx_bnd_osc)):
        if sum(ia) == 0:
            median[i] = np.nan
            error_down[i] = np.nan
            error_upper[i] = np.nan
            continue

        if name == "AIncs":
            d = 1 - d
        
        if name == "Heikki":
            pass
            d = np.log(d)

        median[i] = (np.median(d[ia]))
        error_down[i] = (np.quantile(d[ia], 0.25))
        error_upper[i] = (np.quantile(d[ia], 0.75))

    ax.set_xlabel("Spectral Radius", fontsize=fontsize)
    x = np.linspace(0, 1.5, 100)
    mi, ma = np.min(error_down), np.max(error_upper)
    normalize = lambda d: (d - mi) / (ma - mi)
    ax.fill_between(x, normalize(error_down), normalize(error_upper), color='gray', alpha=0.5)
    axs[1, 0].set_ylabel("Measure", fontsize=fontsize)
    ax.set_ylim(-0.1, 1.6)
    ax.plot(x, normalize(median))
    plt.yticks([0, 1], ['min', 'max'])
    print(f"min: ${mi:.2e}}}$ \nmax: ${ma:.2e}}}$".replace("e-0", "e-").replace("e+0", "e+").replace("e",r"\times 10^{"))
    legend = ax.legend(title=f"min: {mi:.2e} \nmax: {ma:.2e}".replace("e+00", "").replace("e-0", "e-").replace("e+0", "e+"))
    plt.setp(legend.get_title(),fontsize=8)
    legend.get_frame().set_alpha(None)
    legend.get_frame().set_facecolor((1, 1, 1, 0.5))
plt.tight_layout()
fig.subplots_adjust(wspace=0, hspace=0)
plt.savefig(f"pictures/spectral_radius_n_nodes_{n_nodes}/lorenz_spectral_radius_dt{"" if not dt_02 else "_02"}.pdf")
    #plt.show()

In [ ]:
fig, axs = plt.subplots(2, 1, figsize=(5, 4), sharex=True, constrained_layout=True)
dt_02 = True
n_nodes = 20
fontsize = 9
quantiles = np.load(f"results/quantiles{"" if not dt_02 else "_02"}.npy")
i_model = 4
Model = models[i_model]
data = np.load(f"results/spectral_radius_n_nodes_{n_nodes}/uncoupled{"" if not dt_02 else "_dt_02"}_{Model(0,0).ode.__class__.__name__}.npz")

n_linspace = 100
names = data["names"][0, 0]
measures = data["measures"][0].T
name2index = { name:i for i, name in enumerate(names)}
name2index["idx"] = 0
n_samples = measures.shape[1] // n_linspace

idx_bnd_osc = np.logical_and(measures[name2index["bounded"]], measures[name2index["ozillating"]]).reshape(n_linspace, n_samples) #measures[name2index["bounded"]].reshape(n_linspace, n_samples).astype(int) #
for i_measure, (i_measure_quantile, name, sname) in enumerate(zip((None,), ('idx',), ("Bounded & Oscillating",))):
    ax = axs[0]
    #ax.set_title(sname)
    ax.set_title("Easter Egg :)", color="white")


    median = np.zeros(100) 
    error_down, error_upper = np.zeros(100), np.zeros(100) 
    for i, (d, ia) in enumerate(zip(measures[name2index[name]].reshape(100, n_samples), idx_bnd_osc)):
        if sum(ia) == 0:
            median[i] = np.nan
            error_down[i] = np.nan
            error_upper[i] = np.nan
            continue

        if name == "AIncs":
            d = 1 - d

        #mi, ma = np.min(d[ia]), np.max(d[ia])
        #d = (d - mi) / (ma - mi)
        if sum(ia) == 0:
            median[i] = np.nan
            error_down[i] = np.nan
            error_upper[i] = np.nan
            continue
        if name == "valid_prediciton_time":
            d *= 0.1 if not dt_02 else 0.02
        if name == "AIncs":
            d = 1 - d
        if name == "mse":
            median[i] = np.median(d[ia])
            error_down[i] = np.quantile(d[ia], 0.25)
            error_upper[i] = np.quantile(d[ia], 0.75)
        elif name == "valid_prediciton_time":
            median[i] = np.median(d)
            error_down[i] = np.quantile(d, 0.25)
            error_upper[i] = np.quantile(d, 0.75)
        elif name == "idx":
            median[i] = np.mean(ia)
            error_down[i] = median[i] 
            error_upper[i] = median[i]
        else: 
            if name == "Heikki":
                threshold = chi2.isf(0.05)
            else:
                threshold = quantiles[-1, i_model, i_measure_quantile]
            test_statistic = (threshold < d[ia])
            median[i] = np.mean(test_statistic)
            var = np.var(test_statistic)
            error_down[i] = np.mean(test_statistic) - var
            error_upper[i] = np.mean(test_statistic) + var

    ax.set_xlabel("spectral radius", fontsize=fontsize)
    x = np.linspace(0, 1.5, 100)
    ax.fill_between(x, error_down, error_upper, color='gray', alpha=0.5, label='variance')
    ax.set_ylabel("Bounded & \nOscillating in %", fontsize=fontsize)
    ax.plot(x, median, label='mean')
    #ax.legend()

    # plot lower row
    ax = axs[1]
    name = "valid_prediciton_time"
    sname = "VPT"

    median = np.zeros(100) 
    error_down, error_upper = np.zeros(100), np.zeros(100) 
    for i, (d, ia) in enumerate(zip(measures[name2index[name]].reshape(100, n_samples), idx_bnd_osc)):
        if sum(ia) == 0:
            median[i] = np.nan
            error_down[i] = np.nan
            error_upper[i] = np.nan
            continue
        if name == "valid_prediciton_time":
            d *= 0.1 if not dt_02 else 0.02
        if name == "AIncs":
            d = 1 - d
        
        if name == "Heikki":
            pass
            #d = np.log(d)

        median[i] = (np.median(d[ia]))
        error_down[i] = (np.quantile(d[ia], 0.25))
        error_upper[i] = (np.quantile(d[ia], 0.75))

    ax.set_xlabel("Spectral Radius", fontsize=fontsize)
    x = np.linspace(0, 1.5, 100)
    normalize = lambda d: d #(d - mi) / (ma - mi)
    ax.fill_between(x, normalize(error_down), normalize(error_upper), color='gray', alpha=0.5)
    ax.set_ylabel("VPT", fontsize=fontsize)
    #ax.set_ylim(-0.1, 1.1)
    ax.plot(x, normalize(median))
    #plt.yticks([0, 1], ['min', 'max'])


plt.tight_layout()
fig.subplots_adjust(hspace=0)

plt.savefig(f"pictures/spectral_radius_n_nodes_{n_nodes}/lorenz_vpt_dt{"" if not dt_02 else "_02"}.pdf")
    #plt.show()

In [ ]:
cm = 1/2.54
fig, axs = plt.subplots(3, 4, figsize=((21 - 5)*cm*4/5, 3), sharey='row', sharex=True)
dt_02 = True
n_nodes = 20
fontsize = 9
plt.rcParams.update({'font.size': fontsize})
quantiles = np.load(f"results/quantiles{"" if not dt_02 else "_02"}.npy")
i_model = 4
Model = models[i_model]
data = np.load(f"results/spectral_radius_n_nodes_{n_nodes}/uncoupled{"" if not dt_02 else "_dt_02"}_{Model(0,0).ode.__class__.__name__}.npz")

n_linspace = 100
names = data["names"][0, 0]
measures = data["measures"][0].T
name2index = { name:i for i, name in enumerate(names)}
name2index["idx"] = 0
n_samples = measures.shape[1] // n_linspace
idx_bnd_osc = np.logical_and(measures[name2index["bounded"]], measures[name2index["ozillating"]]).reshape(n_linspace, n_samples) #measures[name2index["bounded"]].reshape(n_linspace, n_samples).astype(int) #
for i_measure, (i_measure_quantile, name, sname) in enumerate(zip((3, 1, 2, 0), ('Heikki', 'Adev', 'Ddev', 'AIncs'), ("Log GCI", "ADev", "TVar", "AExc"))):
    ax = axs[0, i_measure]
    ax.set_title(sname, fontsize=fontsize)

    median = np.zeros(100) 
    error_down, error_upper = np.zeros(100), np.zeros(100) 
    for i, (d, ia) in enumerate(zip(measures[name2index[name]].reshape(100, n_samples), idx_bnd_osc)):
        if sum(ia) == 0:
            median[i] = np.nan
            error_down[i] = np.nan
            error_upper[i] = np.nan
            continue

        if name == "AIncs":
            d = 1 - d

        #mi, ma = np.min(d[ia]), np.max(d[ia])
        #d = (d - mi) / (ma - mi)

        if name == "Heikki":
            threshold = chi2.isf(0.05)
        else:
            threshold = quantiles[-1, i_model, i_measure_quantile]

        if name == "AIncs":
            d = 1 - d
            threshold = 1 - threshold
        test_statistic = (threshold < d[ia])
        median[i] = np.mean(test_statistic)
        var = np.var(test_statistic)
        error_down[i] = np.mean(test_statistic) - var
        error_upper[i] = np.mean(test_statistic) + var

    ax.set_xlabel("spectral radius", fontsize=fontsize)
    x = np.linspace(0, 1.5, 100)
    ax.fill_between(x, error_down, error_upper, color='gray', alpha=0.5)
    axs[0, 0].set_ylabel("Rejected in %", fontsize=fontsize)
    ax.plot(x, median)


    # plot lower row
    ax = axs[1, i_measure]

    median = np.zeros(100) 
    error_down, error_upper = np.zeros(100), np.zeros(100) 
    for i, (d, ia) in enumerate(zip(measures[name2index[name]].reshape(100, n_samples), idx_bnd_osc)):
        if sum(ia) == 0:
            median[i] = np.nan
            error_down[i] = np.nan
            error_upper[i] = np.nan
            continue

        if name == "AIncs":
            d = 1 - d
        
        if name == "Heikki":
            pass
            d = np.log(d)

        median[i] = (np.median(d[ia]))
        error_down[i] = (np.quantile(d[ia], 0.25))
        error_upper[i] = (np.quantile(d[ia], 0.75))

    ax.set_xlabel("Spectral Radius", fontsize=fontsize)
    x = np.linspace(0, 1.5, 100)
    mi, ma = np.min(error_down), np.max(error_upper)
    normalize = lambda d: (d - mi) / (ma - mi)
    ax.fill_between(x, normalize(error_down), normalize(error_upper), color='gray', alpha=0.5)
    axs[1, 0].set_ylabel("Measure", fontsize=fontsize)
    ax.set_ylim(-0.1, 1.6)
    ax.plot(x, normalize(median))
    ax.set_yticks([0, 1], ['min', 'max'])
    print(f"min: ${mi:.2e}}}$ \nmax: ${ma:.2e}}}$".replace("e-0", "e-").replace("e+0", "e+").replace("e",r"\times 10^{"))
    legend = ax.legend(title=f"min: {mi:.2e} \nmax: {ma:.2e}".replace("e+00", "").replace("e-0", "e-").replace("e+0", "e+"))
    plt.setp(legend.get_title(),fontsize=8)
    legend.get_frame().set_alpha(None)
    legend.get_frame().set_facecolor((1, 1, 1, 0.5))


    # plot measure of only accepted samples
    ax = axs[2, i_measure]

    median = np.zeros(100) 
    error_down, error_upper = np.zeros(100), np.zeros(100) 
    for i, (d, ia) in enumerate(zip(measures[name2index[name]].reshape(100, n_samples), idx_bnd_osc)):
        if sum(ia) == 0:
            median[i] = np.nan
            error_down[i] = np.nan
            error_upper[i] = np.nan
            continue

        if name == "AIncs":
            d = 1 - d
        
        if name == "Heikki":
            pass
            d = np.log(d)

        if name == "Heikki":
            threshold = np.log(chi2.isf(0.05))
        else:
            threshold = quantiles[-1, i_model, i_measure_quantile]

        if name == "AIncs":
            d = 1 - d
            threshold = 1 - threshold
        test_statistic = d[ia][(threshold > d[ia])]

        if (len(test_statistic) == 0):
            test_statistic = np.nan
        median[i] = (np.median(test_statistic))
        error_down[i] = (np.quantile(test_statistic, 0.25))
        error_upper[i] = (np.quantile(test_statistic, 0.75))

    ax.set_xlabel("Spectral Radius", fontsize=fontsize)
    x = np.linspace(0, 1.5, 100)
    mi, ma = np.min(error_down), np.max(error_upper)
    normalize = lambda d: (d - mi) / (ma - mi)
    ax.fill_between(x, normalize(error_down), normalize(error_upper), color='gray', alpha=0.5)
    axs[2, 0].set_ylabel("Measure \n Accepted", fontsize=fontsize)
    ax.set_ylim(-0.1, 1.6)
    ax.plot(x, normalize(median))
    plt.yticks([0, 1], ['min', 'max'])
    print(f"min: ${mi:.2e}}}$ \nmax: ${ma:.2e}}}$".replace("e-0", "e-").replace("e+0", "e+").replace("e",r"\times 10^{"))
    legend = ax.legend(title=f"min: {mi:.2e} \nmax: {ma:.2e}".replace("e+00", "").replace("e-0", "e-").replace("e+0", "e+"))
    plt.setp(legend.get_title(),fontsize=8)
    legend.get_frame().set_alpha(None)
    legend.get_frame().set_facecolor((1, 1, 1, 0.5))
plt.tight_layout()
fig.subplots_adjust(wspace=0, hspace=0)
plt.savefig(f"pictures/spectral_radius_n_nodes_{n_nodes}/lorenz_spectral_radius_dt{"" if not dt_02 else "_02"}.pdf")
    #plt.show()

In [ ]:
from tabulate import tabulate
import numpy as np
import matplotlib.pyplot as plt


dt_02 = True
n_nodes = 20
quantiles = np.load(f"results/quantiles{"" if not dt_02 else "_02"}.npy")

for i_model, Model in enumerate(models):
    data = np.load(f"results/spectral_radius_n_nodes_{n_nodes}/uncoupled{"" if not dt_02 else "_dt_02"}_{Model(0,0).ode.__class__.__name__}.npz")

    n_linspace = 100
    names = data["names"][0, 0]
    measures = data["measures"][0].T
    
    measures[name2index["AIncs"]] = 1 - measures[name2index["AIncs"]] 

    name2index = { name:i for i, name in enumerate(names)}
    name2index["idx"] = 0
    n_samples = measures.shape[1] // n_linspace
    idx_bnd_osc = np.logical_and(measures[name2index["bounded"]], measures[name2index["ozillating"]]).reshape(n_linspace, n_samples) #measures[name2index["bounded"]].reshape(n_linspace, n_samples).astype(int) #
    array = np.column_stack((('AExc', 'ADev', 'TVar', 'GCI',),  np.corrcoef(measures[-4:,][(3, 1, 2, 0), :].reshape(4, 100, n_samples)[:, idx_bnd_osc])))
    latex_table = tabulate(array, tablefmt="latex", headers=('Correlation', 'AExc', 'ADev', 'TVar', 'GCI'))

    print(latex_table)

In [ ]:
models_name = ["Sprott B", "Sprott C", "Sprott D", "Roessler76", "Lorenz63"]
thresholds = np.load(f"results/reference_values_ranges{"_02" if dt_02 else ""}.npy")
thresholds = thresholds[:, i_model]
thresholds[:, 0] = 1 - thresholds[:, 0]
thresholds = np.quantile(thresholds, 0.95, axis=0)
thresholds[-1] = chi2.ppf(0.95)

In [ ]:

for i_model, Model in enumerate(models):
    data = np.load(f"results/spectral_radius_n_nodes_{n_nodes}/uncoupled{"" if not dt_02 else "_dt_02"}_{Model(0,0).ode.__class__.__name__}.npz")

    n_linspace = 100
    names = data["names"][0, 0]
    measures = data["measures"][0].T
    
    measures[name2index["AIncs"]] = 1 - measures[name2index["AIncs"]] 

    name2index = { name:i for i, name in enumerate(names)}
    name2index["idx"] = 0
    n_samples = measures.shape[1] // n_linspace
    idx_bnd_osc = np.logical_and(measures[name2index["bounded"]], measures[name2index["ozillating"]]).reshape(n_linspace, n_samples) #measures[name2index["bounded"]].reshape(n_linspace, n_samples).astype(int) #
    array = np.column_stack((('AExc', 'ADev', 'TVar', 'GCI',),  np.corrcoef(measures[-4:,][(3, 1, 2, 0), :].reshape(4, 100, n_samples)[:, idx_bnd_osc] > thresholds[:, None])))
    latex_table = tabulate(array, tablefmt="latex", headers=('Correlation', 'AExc', 'ADev', 'TVar', 'GCI'))

    print(latex_table)

In [ ]:
from itertools import combinations


def mean_deviation(array):
    out = np.ones([len(array)]*2)
    array = array.astype("int")
    for i,j in combinations(range(len(array)), 2):
        merror = np.mean(np.absolute(array[i] - array[j]))
        out[i, j] = out[j, i] = merror 
    return out

In [ ]:

for i_model, Model in enumerate(models[-1:]):
    data = np.load(f"results/spectral_radius_n_nodes_{n_nodes}/uncoupled{"" if not dt_02 else "_dt_02"}_{Model(0,0).ode.__class__.__name__}.npz")

    n_linspace = 100
    names = data["names"][0, 0]
    measures = data["measures"][0].T
    
    measures[name2index["AIncs"]] = 1 - measures[name2index["AIncs"]] 

    name2index = { name:i for i, name in enumerate(names)}
    name2index["idx"] = 0
    n_samples = measures.shape[1] // n_linspace
    idx_bnd_osc = np.logical_and(measures[name2index["bounded"]], measures[name2index["ozillating"]]).reshape(n_linspace, n_samples) #measures[name2index["bounded"]].reshape(n_linspace, n_samples).astype(int) #
    array = np.column_stack((('AExc', 'ADev', 'TVar', 'GCI',),  mean_deviation(measures[-4:,][(3, 1, 2, 0), :].reshape(4, 100, n_samples)[:, idx_bnd_osc] > thresholds[:, None])))
    latex_table = tabulate(array, tablefmt="latex", headers=('Correlation', 'AExc', 'ADev', 'TVar', 'GCI'))

    print(latex_table)